In [1]:
import scraping
from scraping import scraper
import pandas as pd
import ast
import llm

In [2]:
topics_urls = []
base_topic_url = "https://www.webmd.com/a-to-z-guides/health-topics?pg="
for num in range(97, 97 + 26):
    topics_urls.append(base_topic_url + chr(num))

In [3]:
'''
for i in range(len(topics_urls)):
    webscraper = scraper(topics_urls[i:i+1], "./scraped/out_"+chr(97+i)+".csv")
    webscraper.scrape()
'''

'\nfor i in range(len(topics_urls)):\n    webscraper = scraper(topics_urls[i:i+1], "./scraped/out_"+chr(97+i)+".csv")\n    webscraper.scrape()\n'

In [4]:
df = pd.read_csv("scraped/webmd_diseases.csv")
df.loc[0, "body"]

'["What Is AAT Deficiency? Alpha-1 antitrypsin (AAT) deficiency is a condition that can cause serious lung or liver disease. Symptoms often include trouble breathing and jaundiced, or yellow, skin. It\'s a genetic disease, which means it\'s passed down to you from your parents. You get it because your liver doesn\'t make enough of the AAT protein, and much of what it does make gets trapped in your liver. You need this protein to protect your lungs from inflammation and damage caused by infections and irritants such as smoke and pollution. If your lungs are damaged, you can get COPD or emphysema. AAT deficiency is sometimes known as genetic COPD or genetic emphysema. And when AAT\\xa0builds up in your liver, the buildup may cause scarring, or cirrhosis, of your liver. You might not know you have the disease until you\'re an adult. Most people get their first symptoms between the ages of 20 and 50.", "Symptoms of AAT Deficiency When you have AAT deficiency, your lungs don\'t get enough o

In [5]:
def parse(text):
    text = text[1:-1].replace("\\xa0", ' ').replace("\",", "|").replace("',", "|").replace("\\", "").split("|")
    for i in range(len(text)):
        text[i] = text[i].strip(" \"'")
    return text

In [6]:
def df_with_map_text_chunk_to_index(df):
    mapped = pd.DataFrame(columns=["df_index", "text"])
    for i in range(df.shape[0]):
        parsed = parse(df.loc[i, "body"])
        article_row = []
        for chunk in parsed:
            new_row = {"df_index": i, "text": chunk}
            article_row.append(new_row)
        new_df = pd.DataFrame(article_row)
        mapped = pd.concat([mapped, new_df])
    return mapped

In [7]:
def create_lookup(df):
    mapped = pd.DataFrame(columns=["text"])
    for i in range(df.shape[0]):
        parsed = parse(df.loc[i, "body"])
        temp = ""
        for chunk in parsed:
            temp += chunk
        mapped = pd.concat([mapped, pd.DataFrame({"text": temp}, index=[i])])
    return mapped

In [8]:
#p = df_with_map_text_chunk_to_index(df)
#p.shape

In [9]:
#test = pd.read_csv("scraped/chunked_embedding.csv", converters={"embedding": ast.literal_eval})


In [10]:
#val = test.loc[:, 'embedding'].values
#for l in val:
#    print(len(l))

In [11]:
emb_df, emb_np = llm.load_data("./scraped/embeddings.csv")
lookup = pd.read_csv("./scraped/lookup.csv")

In [12]:
lookup.shape, emb_df.shape, emb_np.shape

((1205, 1), (12299, 2), (12299, 1536))

In [13]:
lookup.head()

,text
0,What Is AAT Deficiency? Alpha-1 antitrypsin (A...
1,"Headache and nausea are common, and they can h..."
2,Amyloidosis is when an abnormal protein called...
3,"What Is a Pap Smear? A Pap smear, also called ..."
4,There are a few types of procedures to induce ...


In [14]:
emb_df.head()

,df_index,embedding
0,0,"[-0.0028367843478918076, -0.03506242111325264,..."
1,0,"[0.008348198607563972, -0.034819234162569046, ..."
2,0,"[-0.01567361317574978, 0.0023761484771966934, ..."
3,0,"[-0.021367991343140602, -0.023144785314798355,..."
4,0,"[-0.016345791518688202, -0.026688482612371445,..."
